In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
import joblib
import pickle


In [3]:
df = pd.read_csv("WELFake_Dataset.csv")


In [4]:
df=df.drop(["title"] ,axis=1)

In [5]:
df.isnull().sum()

Unnamed: 0     0
text          39
label          0
dtype: int64

In [6]:
df = df.dropna(subset=['text'])
df.reset_index(inplace=True)

In [7]:
df.isnull().sum()

index         0
Unnamed: 0    0
text          0
label         0
dtype: int64

In [8]:
df = df.drop(['index','Unnamed: 0'],axis=1)

In [9]:
df = df.sample(frac = 1)

In [10]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [11]:
df["text"] = df["text"].apply(wordopt)

In [67]:
df

,text,label
62144,beirut reuters israeli missiles struck a m...,0
36546,in the height of the summer travel season bri...,0
32343,patrick henningsen century wirethis week some...,1
5349,doha reuters qatar will buy fighter jets a...,0
4481,baghdad reuters british prime minister the...,0
...,...,...
31030,kiev reuters ukrainian president petro por...,0
3022,geneva reuters the united nations human ri...,0
46235,syrian war report november isis and al n...,1
37443,here s how against the republican establishmen...,1


In [13]:
x = df["text"]
y = df["label"]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
joblib.dump(vectorization, 'vectorization.pkl')
xv_test = vectorization.transform(x_test)


In [16]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LRModel = LR.fit(xv_train,y_train)

c:\Users\loyal\Desktop\ML\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
pred_lr=LR.predict(xv_test)

In [18]:
LR.score(xv_test, y_test)

0.9478465913031416

In [19]:

pickle.dump(LRModel, open('LRmodel.pkl','wb'))

In [22]:

loaded_model=joblib.load("LRmodel.pkl")


In [28]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = loaded_model.predict(new_xv_test)
    # pred_DT = DT.predict(new_xv_test)
    # pred_GBC = GBC.predict(new_xv_test)
    # pred_RFC = RFC.predict(new_xv_test)
    return print("\n\nLR Prediction: {}".format(output_lable(pred_LR[0])))
    #return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),output_lable(pred_DT[0]),output_lable(pred_GBC[0]),output_lable(pred_RFC[0])))



LR Prediction: Not A Fake News


In [70]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DTmodel =DT.fit(xv_train, y_train)

In [71]:
pred_dt = DT.predict(xv_test)

In [72]:
DT.score(xv_test, y_test)

0.9201747694014841

In [73]:
pickle.dump(DTmodel, open('DTmodel.pkl','wb'))

In [77]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [78]:
pred_rfc = RFC.predict(xv_test)

In [79]:
RFC.score(xv_test, y_test)

0.9271100631111727

<h3>Result</h3>


Logistic Regression model gives the highest accuracy